**改編自〈sovits3.0一键脚本（小狼躺平了，所以是深夜诗人修改版本,已更新32k/48k分支切换）.ipynb〉**

https://colab.research.google.com/drive/1rCUOOVG7-XQlVZuWRAj5IpGrMM8t07pE?usp=sharing

做了大幅(?)調整使其可用在vast.ai的機台上頭（運作映像為nvidia/cuda:12.0.1-runtime-ubuntu22.04 `00602486357e8c88c07157ddcd8fb2d7`），包含各種版本差異造成的坑。

原Repo `innnky/so-vits-svc` 因為某些因素不見了，所以使用我自己的fork，預訓練模型採用 `therealvul/so-vits-svc-4.0-init` 提供的模型。

註解繁中化。

In [1]:
# 查看顯示卡
!nvidia-smi

Tue Mar  7 09:20:33 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:03:00.0 Off |                  N/A |
|  0%   42C    P8     4W / 420W |      1MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from pathlib import Path

root = Path('/root')
sovits = root / 'so-vits-svc'

In [14]:
#@title Clone github Repo
!git clone https://github.com/gimi65536/so-vits-svc -b 4.0
%cd {sovits}

Cloning into 'so-vits-svc'...
remote: Enumerating objects: 983, done.
remote: Counting objects: 100% (975/975), done.
remote: Compressing objects: 100% (478/478), done.
remote: Total 983 (delta 489), reused 921 (delta 462), pack-reused 8
Receiving objects: 100% (983/983), 25.47 MiB | 20.28 MiB/s, done.
Resolving deltas: 100% (489/489), done.
/root/so-vits-svc


In [ ]:
# 安裝Dependencies
!apt-get install build-essential python3.10-dev zip unzip -y
!pip install pyworld praat-parselmouth librosa==0.9.2 fairseq matplotlib

In [3]:
# 下載必要模型文件
!wget -P hubert/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/checkpoint_best_legacy_500.pt

--2023-03-12 10:27:45--  https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/checkpoint_best_legacy_500.pt
Resolving huggingface.co (huggingface.co)... 52.5.232.28, 54.146.163.17, 34.206.141.115, ...
Connecting to huggingface.co (huggingface.co)|52.5.232.28|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/c7/21/c7214c5855c8c13ab3ad357ebc651593e9ca56b492469e678bdbda597a830f99/60d936ec5a566776fc392e69ad8b630d14eb588111233fe313436e200a7b187b?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27checkpoint_best_legacy_500.pt%3B+filename%3D%22checkpoint_best_legacy_500.pt%22%3B&Expires=1678876067&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2M3LzIxL2M3MjE0YzU4NTVjOGMxM2FiM2FkMzU3ZWJjNjUxNTkzZTljYTU2YjQ5MjQ2OWU2NzhiZGJkYTU5N2E4MzBmOTkvNjBkOTM2ZWM1YTU2Njc3NmZjMzkyZTY5YWQ4YjYzMGQxNGViNTg4MTExMjMzZmUzMTM0MzZlMjAwYTdiMTg3Yj9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l

# 資料集預處理預處理

該處理腳本可以一次性預處理多位角色，並且生成多角色的filelists以及對應的配置文件

只需要將你的資料集按照以下文件結構放到dataset_raw資料夾下


```
dataset_raw
├───speaker0
│   ├───xxx1-xxx1.wav
│   ├───...
│   └───Lxx-0xx8.wav
└───speaker1
    ├───xx2-0xxx2.wav
    ├───...
    └───xxx7-xxx007.wav

```




In [4]:
# 設定訓練資料集備份與還原的路徑，預設為/root
DATA_PATH = root

In [ ]:
# 資料集名稱（壓縮檔名稱不帶zip）
DATASETNAME = "michiru"  #@param {type:"string"}
# 壓縮檔路徑
ZIP_PATH = root
ZIP_NAME = ZIP_PATH / DATASETNAME

!unzip -d {sovits / 'dataset_raw'} {ZIP_NAME}.zip

In [ ]:
# 重取樣到44100Hz生成配置文件
!python3 resample.py

In [ ]:
# 劃分訓練集、
!python3 preprocess_flist_config.py

In [ ]:
# 自訂config...

In [ ]:
# 生成hubert和f0
!python3 preprocess_hubert_f0.py

In [ ]:
# 至此，數據集預處理製作完畢，將資料集和相關文件保存到so-vits-svc資料夾中並自行下載，方便下次訓練使用

# 壓縮dataset資料夾
!zip -r dataset.zip dataset
# 自訂壓縮檔名稱（不含zip）
dataset_name_drive = "44kmichiru_dataset"
DATASET_PATH_DRIVE = DATA_PATH / dataset_name_drive
!mkdir -p {DATASET_PATH_DRIVE}

!cp dataset.zip "{DATASET_PATH_DRIVE}"
!cp configs/config.json "{DATASET_PATH_DRIVE}"
!cp filelists/train.txt "{DATASET_PATH_DRIVE}"
!cp filelists/test.txt "{DATASET_PATH_DRIVE}"
!cp filelists/val.txt "{DATASET_PATH_DRIVE}"

# 自行下載壓縮檔{DATA_PATH / dataset_name_drive}.zip

In [4]:
# 已經預處理過資料集的話，就可以跳過預處理部分，直接解壓縮處理過的資料以及配置文件

# 壓縮檔名稱（不含zip）
back_up_name = "44kmika_dataset"
BACK_UP_DATASET_PATH = DATA_PATH / back_up_name
!unzip {BACK_UP_DATASET_PATH / 'dataset.zip'} -d .
!cp {BACK_UP_DATASET_PATH / 'config.json'} configs/config.json 
!cp {BACK_UP_DATASET_PATH / 'val.txt'} filelists/val.txt
!cp {BACK_UP_DATASET_PATH / 'train.txt'} filelists/train.txt
!cp {BACK_UP_DATASET_PATH / 'test.txt'} filelists/test.txt

# 將之前訓練過的模型G_xx.pth與D_xx.pth放到 logs/44k 底下……

Archive:  /root/48kmika_dataset/dataset.zip
replace /content/so-vits-svc/dataset/48k/mika_voice/183845.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: ^C


# 訓練

In [21]:
# 如果是首次訓練，則需事先下載預訓練模型
!wget -P logs/44k/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/G_0.pth
!wget -P logs/44k/ https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/D_0.pth

--2023-03-12 11:01:32--  https://huggingface.co/therealvul/so-vits-svc-4.0-init/resolve/main/G_0.pth
Resolving huggingface.co (huggingface.co)... 34.196.181.113, 54.88.208.113, 34.230.94.181, ...
Connecting to huggingface.co (huggingface.co)|34.196.181.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/c7/21/c7214c5855c8c13ab3ad357ebc651593e9ca56b492469e678bdbda597a830f99/20a327c54e5731bed377bd38404bc32ab98e66a1b2777b0af4cc034d4d6914b0?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27G_0.pth%3B+filename%3D%22G_0.pth%22%3B&Expires=1678877760&Policy=eyJTdGF0ZW1lbnQiOlt7IlJlc291cmNlIjoiaHR0cHM6Ly9jZG4tbGZzLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2M3LzIxL2M3MjE0YzU4NTVjOGMxM2FiM2FkMzU3ZWJjNjUxNTkzZTljYTU2YjQ5MjQ2OWU2NzhiZGJkYTU5N2E4MzBmOTkvMjBhMzI3YzU0ZTU3MzFiZWQzNzdiZDM4NDA0YmMzMmFiOThlNjZhMWIyNzc3YjBhZjRjYzAzNGQ0ZDY5MTRiMD9yZXNwb25zZS1jb250ZW50LWRpc3Bvc2l0aW9uPSoiLCJDb25kaXRpb24iOnsiRGF0ZUxlc3NUaGFuIjp7IkFXUzpFcG9j

In [ ]:
!python3 train.py -c configs/config.json -m 44k